## Neural Network

### Introduction to Perceptron 

* Artificial Neural Network (ANN) actually have a basis in biology.
* Perceptron is mimic biological neurons with artificial neuron.
* The simple model of perceptron
    
           (12)
        input 0 \
                 \ weight 0 (0.5)
           (+1)   \  12*0.5 = 6
         bias ---- ( activation funtion ) ---- output
                  /     6 - 4 + 1 = 3           (1)
                 / weight 1(-1)
        input 1 /    4 * (-1) = -4
            (4)
            
        - inputs will be values of features
        - weight initialy start as randome
          inputs multiply by weights then passed result to ac activation function
        - the simple activation function:
            if sum of inputs is positive return 1
            if sum of inputs us negative return 0
        - there is possible issue, if the original inputs start of zeros?
            then any weight multiply by inputs always zero!
        - bias term adding to fix these possuble issue
        
* Mathematically representation:

    y = Sum(x<sub>i</sub>w<sub>i</sub>) + b


### Activation Functions
* Simple activation function unit step
* Dynamic function : Sigmoid function  output 0 - 1
    
    f(x) = 1/(1 + e<sup>-(x)</sup>)
    
    where : z = xw + b
* Hyperbolic Tangent: tanh(z) output (-1) - 1
    
    cosh z = 1/2*(e<sup>z</sup> + e<sup>-z</sup>)
    
    sinh z = 1/2*(e<sup>z</sup> - e<sup>-z</sup>)
    
    tanh z = sinh x / cosh z
    
    where : z = xw + b
* Rectified Linear Unit (ReLU): output max(0, z)
     
     where : z = xw + b
     

### Cost Functions
Cost function use to evaluate the performance of a neuron
by measure how far off from the expedted value
We'll use the following variables:
    - y to represent the true value
    - a to represent the neuron's prediction
    in terms of weights and bias
        z = wx + b
    - pass z into sigmoid activation function Q(z) = a 
    
#### Quadratic Cost:
   C = Sum(y - a)<sup>2</sup> / n
   
    - this calculation ca cause a slowdown in our learning speed
    - we can see the larger errors are more prominent du to the square.

#### Cross Entropy:
   C = (-1/n) * Sum(y*ln(a) + (1-y)*ln(1-a))
    - this cost function allows for faster learning
    

### Operation class
- Input nodes
- Output nodes
- Complute operation


In [1]:
class Operation():
    """
    An Operation is a node in a "Graph". TensorFlow will also use this concept of a Graph.
    
    This Operation class will be inherited by other classes that actually compute the specific
    operation, such as adding or matrix multiplication.
    """
    
    def __init__(self, input_nodes = []):
        """
        Intialize an Operation
        """
        self.input_nodes = input_nodes # The list of input nodes
        self.output_nodes = [] # List of nodes consuming this node's output
        
        # For every node in the input, we append this operation (self) to the list of
        # the consumers of the input nodes
        for node in input_nodes:
            node.output_nodes.append(self)
        
        # There will be a global default graph (TensorFlow works this way)
        # We will then append this particular operation
        # Append this operation to the list of operations in the currently active default graph
        _default_graph.operations.append(self)
  
    def compute(self):
        """ 
        This is a placeholder function. It will be overwritten by the actual specific operation
        that inherits from this class.
        
        """
        
        pass

## Example Operations
### Addition

In [2]:

class Add(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x + y



### Multiplication

In [3]:
class Multiply(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x * y
    
    

### Matrix Multiplication

In [4]:
class Matmul(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x.dot(y)

#### Placeholder:
* An empty node that needs a values to be provided to compute output.

#### Variable:
* Changeable parameter of graph.

#### Graph:
- An variable conectiong variables and placeholders to operations.


## Placeholders

In [5]:
class Placeholder():
    """
    A placeholder is a node that needs to be provided a value for computing the output in the Graph.
    """
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

## Variables

In [6]:
class Variable():
    """
    This variable is a changeable parameter of the Graph.
    """
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
         
        _default_graph.variables.append(self)


In [7]:
class Graph():
    
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        """
        Sets this Graph instance as the Global Default Graph
        """
        global _default_graph
        _default_graph = self

## A Basic Graph

$$ z = Ax + b $$

With A=10 and b=1

$$ z = 10x + 1 $$

Just need a placeholder for x and then once x is filled in we can solve it!

In [8]:
# example
g = Graph()

In [9]:
g.set_as_default()

In [10]:
A = Variable(10)

In [11]:
b = Variable(1)

In [12]:
x = Placeholder()

In [13]:
y = Multiply(A, x)

In [14]:
z = Add(y, b)

## Session
- In order to execute all the opeation within the nodes, we need to execute all the operations within a Session
- The "Post order traversal" will use to make sure all execute the node in correction order.

### Traversing Operation Nodes

In [15]:
def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b).
    """
    nodes_postorder = []
    def postorder(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                postorder(input_node)
        nodes_postorder.append(node)
            
    postorder(operation)
    return nodes_postorder

In [16]:
import numpy as np

In [17]:
class Session(object):
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output

In [18]:
sess = Session()

In [19]:
result = sess.run(operation=z,feed_dict={x:10})

In [20]:
result

101

In [21]:
g = Graph()

In [22]:
g.set_as_default()

In [23]:
A = Variable([[10, 20], [30, 40]])

In [24]:
b = Variable([1, 1])

In [25]:
x = Placeholder()

In [26]:
y = Matmul(A, x)

In [27]:
z = Add(y, b)

In [28]:
sess = Seesion()

NameError: name 'Seesion' is not defined

In [ ]:
sess.run(z, {x:10})

## Activation Function

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
sample_z = np.linspace(-10, 10, 100)

In [ ]:
sample_a = sigmoid(sample_z)


In [ ]:
plt.plot(sample_z, sample_a)

In [ ]:
class Sigmoid(Operation):
    
    def __init__(self, z):
        super().__init__([z])
        
    def compute(self, z):
        return 1 / (1 + np.exp(-z))

## Classification Example

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
data = make_blobs(n_samples=50, n_features=2, centers=2, random_state=75)

In [ ]:
data

In [ ]:
features = data[0]
labels = data[1]

In [ ]:
plt.scatter(features[:,0], features[:,1], c=labels, cmap='coolwarm')

In [ ]:
# DRAW A LINE THAT SEPERATES CLASSES
x = np.linspace(0,11,10)
y = -x + 5
plt.scatter(features[:,0],features[:,1],c=labels,cmap='coolwarm')
plt.plot(x,y)

### Defining the Perceptron
    𝑦=𝑚𝑥+𝑏
    𝑦=−𝑥+5
    𝑓1=𝑚𝑓2+𝑏,𝑚=1
    𝑓1=−𝑓2+5
    𝑓1+𝑓2−5=0

### Convert to a Matrix Representation of Features
Features
$$ w^Tx + b = 0 $$
​
$$   \Big(1, 1\Big)f - 5 = 0 $$
​
Then if the result is > 0 its label 1, if it is less than 0, it is label=0
​
​


### Example Point

Let's say we have the point f1=2 , f2=2 otherwise stated as (8,10). Then we have:

$$ 
\begin{pmatrix} 
1 , 1
\end{pmatrix} 
\begin{pmatrix} 
8 \\
10
\end{pmatrix} + 5 =  $$

In [ ]:
g = Graph()

In [ ]:
g.set_as_default()

In [ ]:
x = Placeholder()

In [ ]:
w = Variable([1, 1])

In [ ]:
b = Variable(-5)

In [ ]:
z = Add(Matmul(w, x), b)

In [ ]:
a = Sigmoid(z)

In [ ]:
sess = Session()

In [ ]:
sess.run(operation=a,feed_dict={x:[8,10]})

In [ ]:
sess.run(operation=a,feed_dict={x:[2,-10]})